# Pivo Recommender

Looking for a new way to compare and recommend beer.  

This notebook scrapes beer reviews from [BeerAdvocate](https://www.beeradvocate.com/) then performs natural language processing on these reviews.  Once a profile of a beer has been created, a similar, semi-similar, or completely different beer can be recommended.

To run the scrapy scraper (and generate all the data required for this analysis) follow these steps:
 1. move into the beerAdvocateScraper directory (cd beerAdvocateScraper)
 2. execute the scraper (scrapy crawl reviewScraper

In [ ]:
# %cd beerAdvocateScraper
# %scrapy crawl reviewScraper

Import and load the necessary libraries.

In [ ]:
import itertools as it
import os
import pandas as pd
import spacy

In [ ]:
nlp = spacy.load('en')

## Phrase Modeling

Phrase modeling is an approach to learning combinations of tokens that together represent meaning multi-word concepts.  These phrase models are developed by looping over the words in the corpus and finding words that appear together more than they should by random chance.  The formula used to determine if whether two tokens $A$ and $B$ constitute a phrase is:

$$
\frac {count(A B) - count_{min}}
{count(A) * count(B)}
*N > threshold
$$

where:
* $count(A)$ is the number of times $A$ appears in the corpus
* $count(B)$ is the number of times $B$ appears in the corpus
* $count(AB)$ is the number of times $AB$ appear in the corpus in that order
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that the phrase appears a minimum number of times
* $threshold$ is a user-defined paramter to control how strong the relationship must be before the two tokens are considered a single concept

Once we have trained the phrase model we can apply it to the reviews in our corpus.  It will consider the multiworded tokens to be single phrases.

The gensim library will help us with phrase modeling, specifically the Phrases class.

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

In [ ]:
%mv beerAdvocateScraper/BeerAdvocateReviews.csv .

In [ ]:
# creating slimReview.csv for testing purposes

!head -20 BeerAdvocateReviews.csv >> slimReviews.csv
!tail -20 BeerAdvocateReviews.csv >> slimReviews.csv

In [ ]:
%mkdir ./intermediate
%mv BeerAdvocateReviews.csv ./intermediate/
%mv slimReviews.csv ./intermediate/

In [ ]:
import re
pattern = re.compile('^[\d,"]+')

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    count = 0
    with open(filename, encoding='utf_8') as f:
        for review in f:
            try:
                yield re.split(pattern, review)[1].replace('\\n', '\n')
            except: pass
            if count % 1000 == 0:
                print(f'on review {count}')
            count += 1
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=1000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [ ]:
intermediate_directory = os.path.join('.', 'intermediate')
# review_txt_filepath = os.path.join(intermediate_directory,'BeerAdvocateReviews.csv')
review_txt_filepath = os.path.join(intermediate_directory,'slimReviews.csv')

%mkdir './intermediate/ngram_all'
ngram_all = os.path.join(intermediate_directory, 'ngram_all')
unigram_sentences_filepath = os.path.join(ngram_all,
                                          'unigram_sentences_all.txt')

We will use `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text.  We will write this data back out to a new file (`unigram_sentence_all`), with one normalized sentence per line.  We will use this data for learning our phrase models.

In [ ]:
with open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
#         print(sentence)
        f.write(sentence + '\n')

Our data in the `unigram_sentences_all` file is now organized as a large text file with one sentence per line.  This format allows us to use gensim's LineSentence class, a convenient iterator for working with gensim's other components.  It *streams* the documents/sentences from disk, so you never have to hold the entire corpus in RAM at once.  This allows you to scale the modeling to a very large corpora.

In [ ]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [ ]:
for unigram_sentence in it.islice(unigram_sentences, 5,7):
    print(u' '.join(unigram_sentence))
    print(u' ')

In [ ]:
bigram_model_file = os.path.join(ngram_all, 'bigram_model')

In [ ]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [ ]:
bigram_sentences_filepath = os.path.join(ngram_all,'bigram_sentences_all.txt')

In [ ]:
with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for unigram_sentence in unigram_sentences:

        bigram_sentence = u' '.join(bigram_model[unigram_sentence])

        f.write(bigram_sentence + '\n')

In [ ]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [ ]:
for bigram_sentence in it.islice(bigram_sentences, 5, 7):
    print(u' '.join(bigram_sentence))
    print(u'')

In [ ]:
trigram_model_filepath = os.path.join(ngram_all, 'trigram_model')

In [ ]:
if 1 == 1:

    phrases_trigram = Phrases(bigram_sentences)
    trigram_model = Phraser(phrases_trigram)
    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

In [ ]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [ ]:
trigram_sentences_filepath = os.path.join(ngram_all, 'trigram_sentences_all.txt')

In [ ]:
with open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for bigram_sentence in bigram_sentences:

        trigram_sentence = u' '.join(trigram_model[bigram_sentence])

        f.write(trigram_sentence + '\n')

In [ ]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [ ]:
for trigram_sentence in it.islice(trigram_sentences, 3, 5):
    print(u' '.join(trigram_sentence))
    print(u'')

In [ ]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:

with open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:

    for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                  batch_size=10000, n_threads=4):

        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in STOP_WORDS]
        
        # remove pronouns
        trigram_review = [term for term in trigram_review
                          if term !='-PRON-']
        

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

In [ ]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 4, 5):
    print(review)

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 4, 5):
        print(review)

In [ ]:
def get_beerID(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    with open(review_txt_filepath, encoding='utf_8') as f:
        for review in f:
            try:
                yield re.split('(^[\d"]+)', review)[1].replace('\\n', '\n')
            except: pass

In [ ]:
from collections import Counter

In [ ]:
beerDict = {}


for i in range(3):
    with open(trigram_reviews_filepath, encoding='utf_8') as f:
        for review in it.islice(f, i, i+1):
            beerID = list(it.islice(get_beerID(review_txt_filepath), i, i+1))[0]
            wordDict = Counter()
            for word in review.split():
                wordDict[word] = wordDict.get(word,0) + 1
#             print(wordDict)
#             print('---------------')
            beerDict[beerID] = beerDict.get(beerID, Counter()) + wordDict
print(beerDict)

In [ ]:
m = pd.read_csv(review_txt_filepath)

beerDict = {}

for i in range(m.shape[0]):
    with open(trigram_reviews_filepath, encoding='utf_8') as f:
        for review in it.islice(f, i, i+1):
            beerID = list(it.islice(get_beerID(review_txt_filepath), i, i+1))[0]
            wordDict = Counter()
            for word in review.split():
                wordDict[word] = wordDict.get(word,0) + 1
            beerDict[beerID] = beerDict.get(beerID, Counter()) + wordDict
# print(beerDict)

In [ ]:
# remove any items in beerDict with less than n occurances

In [ ]:
beerDict.keys()

## Word2Vec

The goal of *word vector embedding models*, or *word vector models* for short is to learn dense numerical representations of each term in a corpus vocabulary.  If the model is succesful, the vectors it learns should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary.  Word vector models are fully unsupervised &mdash; they learn all of these meaning and relationships solely by analyzing the text of the corpus, without any advanced knowledge provided.

In [ ]:
from gensim.models import Word2Vec

# run with or without stop words removed and word lemmatized
trigram_sentences = LineSentence(trigram_reviews_filepath)
# trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

In [ ]:
if 1==1:
    
    # initiate the model and perform one epoch of training
    beer2vec = Word2Vec(trigram_sentences, size=100, window=5,
                       min_count=1, sg=1) #workers=?
    beer2vec.save(word2vec_filepath)
    
    #perform the next n epochs of training
    for i in range(1, 5):
        beer2vec.train(trigram_sentences, total_examples=beer2vec.corpus_count, epochs=2)
        beer2vec.save(word2vec_filepath)
        
        
#load the finished model from disk
beer2vec = Word2Vec.load(word2vec_filepath)
beer2vec.init_sims()

print(f'Trained model for {beer2vec.train_count} epochs.')

In [ ]:

# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in beer2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda tup: -1*tup[-1])

# # unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# # create a DataFrame with the food2vec vectors as data,
# # and the terms as row labels
word_vectors = pd.DataFrame(beer2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors.head(10)


In [ ]:
word_vectors.shape

# Dimension Reduction Using t-SNE 

t-Distributed Stochastic Neighbor Embedding, or *t-SNE*, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets.  It attempts to map high-dimensional data onto a low (2 or 3) dimensional  representation such that the relative distance between points are preserved as closely as possible in both high and low dimensional space.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# from spacy.lang.en.stop_words import STOP_WORDS

tsneInput = word_vectors.drop(STOP_WORDS, errors=u'ignore')
tsneInput = tsneInput.head(400)

In [ ]:
tsneInput.shape

In [ ]:
tsne_filepath = os.path.join(intermediate_directory,
                             u'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     u'tsne_vectors.npy')

In [ ]:
import pickle

In [ ]:
tsneInput.values.shape

In [ ]:
if 1 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsneInput.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsneInput.index),
                            columns=[u'x_coord', u'y_coord'])

In [ ]:
tsne_vectors[u'word'] = tsne_vectors.index

In [ ]:
tsne_vectors.head()

In [ ]:
ids = list(beerDict.keys())
print(ids)

In [ ]:
list1 = []
for i in tsne_vectors.index:
    list1.append(i + '_x')
    list1.append(i + '_y')

In [ ]:
masterDF = pd.DataFrame(columns=list1, index=ids)
masterDF.head()

In [ ]:
masterDF.shape

In [ ]:
for key, value in beerDict.items():
    for k2, v2 in value.items():
        try:
            x_val = tsne_vectors.loc[k2]['x_coord'] * v2
            columnLookupX = k2 + '_x'
            masterDF.loc[key][columnLookupX] = x_val

            y_val = tsne_vectors.loc[k2]['y_coord'] * v2
            columnLookupY = k2 + '_y'
            masterDF.loc[key][columnLookupY] = y_val
        except KeyError:
            pass

In [ ]:
masterDF

In [ ]:
masterDF.fillna(0, inplace=True)

In [ ]:
masterDF

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from itertools import combinations

In [ ]:
list(masterDF.index)

In [ ]:
for pair in combinations(list(masterDF.index), 2):
    cs = 1 - spatial.distance.cosine(masterDF.loc[pair[0]].values.reshape(-1,1), masterDF.loc[pair[1]].values.reshape(-1,1))
    print(f'The similarity between {pair[0]} and {pair[1]} is {cs}')

In [ ]:
# find the similarity between every beer pair
# create a dictionary where the key is the beer and value is the list of beers from most to least similar